<a href="https://colab.research.google.com/github/myutman/NLP/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers

In [0]:
import transformers
from transformers import BertTokenizer, BertModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.functional import F

from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import re

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class QAModel(nn.Module):
    def __init__(self):
        super(QAModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        #self.softmax1 = nn.Softmax(dim=-1)
        #self.softmax2 = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        val = self.bert(x, attention_mask=mask)
        #val = (self.softmax1(val[0]), self.softmax2(val[1]))
        return val

def J(output, actual):
    return - (torch.log(output[0][range(len(output[0])), actual[0][:,0]]) + torch.log(output[1][range(len(output[0])), actual[1]])) / 2

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/NLP/train_qa.csv')

In [0]:
batch_size = 16
max_len = 256

def colate_fn(data):
    #print(len(data))
    X, y, mask = zip(*data)
    #print(torch.tensor(X).shape)
    #print(torch.tensor(y).shape)
    #print(torch.tensor(mask).shape)
    return torch.tensor(X), torch.tensor(y), torch.tensor(mask)

def prepare_dataset(df):
    texts = list(df['paragraph'])
    quests = list(df['question'])
    anss = list(df['answer'])
    X = []
    y = []
    masks = []
    for text, quest, ans in tqdm(list(zip(texts, quests, anss))):
        text = ' '.join(re.findall('\w+', text.lower()))
        quest = ' '.join(re.findall('\w+', quest.lower()))
        ans = ' '.join(re.findall('\w+', ans.lower()))
        text_tokens = tokenizer.encode(text)[1:-1]
        quest_tokens = tokenizer.encode(quest)[1:-1]
        answer_tokens = tokenizer.encode(ans)[1:-1]
        answer_tokens += [tokenizer.pad_token_id] * (max_len - len(answer_tokens))
        while len(text_tokens) > 0:
            cnt = min(len(text_tokens), max_len - len(quest_tokens) - 3)
            tokens = [tokenizer.cls_token_id] + quest_tokens + [tokenizer.sep_token_id] + text_tokens[:cnt] + [tokenizer.sep_token_id] + [tokenizer.pad_token_id] * (max_len - cnt - len(quest_tokens) - 3)
            mask = [0 if (i < len(quest_tokens) + 1) or (tokens[i] == tokenizer.pad_token_id) else 1 for i in range(max_len)]
            text_tokens = text_tokens[cnt:]
            X.append(tokens)
            masks.append(mask)
            y.append(answer_tokens)

    print(np.array(y).shape)
    X_train, X_test, y_train, y_test, mask_train, mask_test = train_test_split(X, y, masks, test_size=0.2)
    train_data = torch.utils.data.DataLoader(list(zip(X_train, y_train, mask_train)), batch_size=batch_size, collate_fn=colate_fn)
    test_data = torch.utils.data.DataLoader(list(zip(X_test, y_test, mask_test)), batch_size=batch_size, collate_fn=colate_fn)
    return train_data, test_data

In [0]:
train_data, test_data = prepare_dataset(df[:200])


(249, 256)


In [0]:
model = QAModel()
model.to(device)

In [0]:
num_epochs = 3
adam = optim.Adam(model.parameters(), lr=5e-5)

kek = False
for i in tqdm(range(num_epochs)):
    for X, y, mask in train_data:
        adam.zero_grad()
        output = model(X.to(device), mask.to(device))
        if not kek:
            print(output)
            kek = True

        loss = 0
        #loss.backward()print(len(text_tokens))
        #adam.step()

(tensor([[[-8.8208e-02, -1.8894e-01,  2.9705e-01,  ...,  3.5803e-01,
           5.4517e-02, -6.3080e-02],
         [-5.0913e-01, -2.3151e-01,  9.8051e-01,  ...,  3.0832e-01,
           4.9233e-02,  8.3002e-02],
         [-4.4870e-01, -5.5196e-01,  1.3694e+00,  ...,  2.0363e-01,
          -5.4608e-02,  1.9301e-01],
         ...,
         [-1.8897e-01, -1.1687e-01,  9.3509e-01,  ...,  3.1623e-01,
           1.3001e-01,  2.2307e-01],
         [-7.4562e-02,  1.3556e-01,  9.4808e-01,  ...,  1.0401e-01,
           3.2027e-02,  7.2050e-02],
         [-4.3954e-01, -1.4931e-01,  2.4023e-01,  ...,  1.3518e-03,
           1.0771e-01,  1.9492e-01]],

        [[-2.6778e-01, -3.0463e-01,  2.2638e-01,  ..., -4.5304e-03,
          -9.1200e-02, -4.5234e-01],
         [-2.1567e-01, -3.1557e-01,  3.5726e-01,  ...,  4.4784e-01,
          -2.4698e-01, -6.5943e-01],
         [-1.7903e-01,  2.1039e-02,  3.7585e-01,  ..., -1.6195e-01,
          -5.6193e-01, -4.7183e-01],
         ...,
         [-2.3256e-01, -

RuntimeError: ignored

In [0]:



question, text = "Где встречаются первые упоминания о строении человеческого тела?", "Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге."
input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_ids = tokenizer.encode(input_text)
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

In [0]:
!nvidia-smi

Sat Dec  7 10:45:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    25W /  75W |   7603MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  